In [1]:
from numba import jit
import pandas as pd
import numpy as np
import warnings
import time
import os
import pickle
import re

warnings.filterwarnings(action='ignore')

In [2]:
amino_acid = {'G':57.02146372376,'A':71.03711378804,'S':87.03202841014,'P':97.05276385232,'V':99.0684139166,
              'T':101.04767847442,'C':103.009185,'L':113.08406398088,'I':113.08406398088,'N':114.04292744752,
              'D':115.02694303224,'Q':128.0585775118,'K':128.09496301826,'E':129.04259309652,'M':131.0404846066,
              'H':137.0589118628,'F':147.0684139166,'R':156.10111102874,'Y':163.0633285387,'W':186.07931295398}


proton = 1.00727647
h2o = 18.0105647
nh3 = 17.026549105
neutron = 1.00235

by_tolerance = 0.25
fragment_tolerance = 0.01

In [103]:
#mass 계산
def residue(sequence):

    nsequence = re.sub('[+-.0-9]', '', sequence)
    num = {}
    total = 0

    for i in range(len(sequence)):
        dot = sequence.find('.', i)
        if sequence[i] == '+' or sequence[i] == '-':
            total += float(sequence[i:dot+7])
            if i == 0:
                num[-1] = float(sequence[i:dot+7])
            elif sequence.find(sequence[i-1]) == i-1:
                n_index = nsequence.find(sequence[i-1])
                num[n_index] = float(sequence[i:dot+7])
            else:
                index_list = [k for k, value in enumerate(sequence[:i-1]) if value == sequence[i-1]]
                same_index = len(index_list)

                n_index_list = [k for k, value in enumerate(nsequence) if value == sequence[i-1]]
                num[n_index_list[same_index]] = float(sequence[i:dot+7])
        elif sequence[i] in amino_acid.keys():
            total += amino_acid[sequence[i]]

    b = []
    y = [total]
    mass = 0
    bmass = 0
    if -1 in num.keys():
        mass += num[-1]
        
    for j in range(len(nsequence)):
        if j in num.keys():
            mass += amino_acid[nsequence[j]]+num[j]
            bmass += amino_acid[nsequence[j]]+num[j]
        else:
            mass += amino_acid[nsequence[j]]
            bmass += amino_acid[nsequence[j]]
        b.append(bmass)
        y.append(total-mass)

    del y[-1]
        
    b = np.array(sorted(b))
    y = np.array(sorted(y))

    return b, y, num

In [96]:
#계산한 mass를 이용하여 theoretical mass(neutral loss 제거) 생성
@jit
def res_ion(b, y, charge):
    
    neutral_loss = np.array([])
    
    temp_b = np.array([])
    for i in range(1,charge):
        temp_b = np.concatenate([temp_b, (b+(proton*i))/i])
        temp_b = np.concatenate([temp_b, ((b+(proton*i))/i)+neutron/i])
        neutral_loss = np.concatenate([neutral_loss, ((b+(proton*i))/i)+neutron/i])
        temp_b = np.concatenate([temp_b, ((b+(proton*i))/i)-h2o/i])
        neutral_loss = np.concatenate([neutral_loss, ((b+(proton*i))/i)-h2o/i])
        temp_b = np.concatenate([temp_b, ((b+(proton*i))/i)-nh3/i])
        neutral_loss = np.concatenate([neutral_loss, ((b+(proton*i))/i)-nh3/i])

    temp_b = np.unique(temp_b)

    temp_y = np.array([])
    for i in range(1,charge):
        temp_y = np.concatenate([temp_y, (y+h2o+(proton*i))/i])
        temp_y = np.concatenate([temp_y, ((y+h2o+proton*i)/i)+neutron/i])
        neutral_loss = np.concatenate([neutral_loss, ((y+h2o+proton*i)/i)+neutron/i])
        temp_y = np.concatenate([temp_y, ((y+h2o+proton*i)/i)-h2o/i])
        neutral_loss = np.concatenate([neutral_loss, ((y+h2o+proton*i)/i)-h2o/i])
        temp_y = np.concatenate([temp_y, ((y+h2o+proton*i)/i)-nh3/i])
        neutral_loss = np.concatenate([neutral_loss, ((y+h2o+proton*i)/i)-nh3/i])

    temp_y = np.unique(temp_y)

    result = np.unique(np.concatenate([temp_b, temp_y]))
    neutral_loss = np.unique(sorted(neutral_loss))

    return result, neutral_loss

In [18]:
@jit
#spec_list: spectrum
#remove하는 이유?: internal fragment ion은 b, y 제거 후에 찾는 것이기 때문에
#internal 찾을때는 res_ion의 result만 써도 됨-result안에 neutral loss가 다 있기때문에
def remove_spec(spec_mzlist, spec_intlist, by):

    cnt = 0
    arr = []

    for i in by:

        if cnt == len(spec_mzlist):
                break

        while True:
            if i-by_tolerance <= spec_mzlist[cnt] and spec_mzlist[cnt] <= i+by_tolerance:
                match = cnt
                k = 1
                while cnt+k<len(spec_mzlist) and spec_mzlist[cnt+k]<=i+by_tolerance:
                    if spec_intlist[match]<spec_intlist[cnt+k]:
                        match = cnt+k
                    else:
                        match = match
                    k += 1
                cnt = match
                
                arr.append(int(cnt))
                cnt += 1
                break

            if i+by_tolerance < spec_mzlist[cnt] :
                break

            cnt += 1

            if cnt == len(spec_mzlist):
                cnt = match if len(arr) != 0 else 0
                break

    tempmz = np.delete(spec_mzlist, arr)
    tempint = np.delete(spec_intlist, arr)

    return tempmz, tempint

In [32]:
def frag_ion(pep, charge, num):
    
    npep = re.sub('[+-.0-9]', '', pep)
    pep_ = npep[1:len(npep)-1]
    
    temp = []
    ind = []
    for char in range(1, charge):
        for n in range(2, len(pep_)+1):
            temp += [pep_[i:i+n]+str(char) for i in range(len(pep_)-n+1)]
            ind += [i+1 for i in range(len(pep_)-n+1)]
            
    temp_mass = []
    append = temp_mass.append

    for i in range(len(temp)):
        tmp_frag = temp[i]
        if -1 in num.keys():
            mass = num[-1]
        else:
            mass = 0
        char = int(tmp_frag[-1])

        for j in range(len(tmp_frag[:-1])):
            if j+ind[i] in num.keys():
                mass += amino_acid[tmp_frag[j]]+num[ind[i]+j]
            else:
                mass += amino_acid[tmp_frag[j]]
        append((mass+(proton*char))/char)

    new_mass = np.array(sorted(temp_mass))
    
    
    internallist = []
    for i in range(len(temp)):
        internallist.append([temp[i], ind[i], temp_mass[i]])
        
    internallist = sorted(internallist, key = lambda x:x[2])
    internalDF = pd.DataFrame(internallist, columns = ['internal', 'ind', 'mass']).drop_duplicates()
    internallist = internalDF.values.tolist()
    new_mass = np.array(internalDF['mass'].values.tolist())
    
    return new_mass, internallist

In [101]:
@jit
def find_spec(spec_mzlist, spec_intlist, theo, neutral_loss, internal):

    tempmz = []
    tempint = []
    cnt = 0
    
    if internal == True:
        tol = fragment_tolerance
    else:
        tol = by_tolerance

    for i in theo:

        if cnt == len(spec_mzlist):
                break
    
        while True:
            if i-tol <= spec_mzlist[cnt] and spec_mzlist[cnt] <= i+tol:
                match = cnt
                k = 1
                while cnt+k<len(spec_mzlist) and spec_mzlist[cnt+k]<=i+tol:
                    if spec_intlist[match]<spec_intlist[cnt+k]:
                        match = cnt+k
                    else:
                        match = match
                    k += 1
                cnt = match
                tempmz.append(i)
                tempint.append(spec_intlist[cnt])
                cnt += 1
                break

            if i+tol < spec_mzlist[cnt] :
                break

            cnt += 1
            if cnt == len(spec_mzlist):
                break

    neutral_index = []
    for i in range(len(tempmz)):
        if tempmz[i] in neutral_loss:
            neutral_index.append(i)

    cnt = 0
    for i in neutral_index:
        del tempmz[i-cnt]
        del tempint[i-cnt]
        cnt += 1
    

    return tempmz, tempint

In [21]:
def internal_fraction(dataset):
    start = time.time()

    mgf_path_dir = 'filtered_library_mgf_files/'
    file_list = os.listdir(mgf_path_dir)
    file_list.sort()

    temp = 0
    length = str(len(dataset))
    data = None
    spec_key = None
    internal_fragment_ion = []

    for idx, (ids, x, y, pept, charge) in enumerate(dataset[['id', 'opt_global_originalfilename', 'nativeID_scan', 'modified_sequence', 'charge']].values):
        if idx%50000 == 0:
            print(str(idx) + ' spectrum', '/', length + ' spectrum')

        d_fn = x
        d_scan = int(y)
        key = (d_fn, d_scan)

        if data == None:
            fn = file_list[temp]
            data = open(mgf_path_dir+fn).readlines()
            ind = 0

        if key == spec_key:

            b_ion, y_ion, num = residue(pept)
            ion_list, neutral_list = res_ion(b_ion, y_ion, int(charge))
            spec_mzarr, spec_intarr = remove_spec(arr_mz, arr_int, ion_list)
            residue_mass, internal_list = frag_ion(pept, int(charge), num)
            find_mass, find_int = find_spec(spec_mzarr, spec_intarr, residue_mass, neutral_list, internal = True)
            find_list = []

            if len(find_mass) != 0:
                k = 0
                while k <= len(internal_list)-1:
                    for j in range(len(find_mass)):
                        temp_list = []
                        if internal_list[k][2] == find_mass[j]:
                            temp_list.append(ids)
                            temp_list.append(pept)
                            temp_list.append(internal_list[k][1])
                            temp_list.append(find_int[j])
                            find_list += [temp_list]
                    k += 1
            else:
                find_list += [[ids, pept, 0, 0]]

            internal_fragment_ion += find_list
            
            continue

        while True:
            if ind == len(data):
                temp += 1
                fn = file_list[temp]
                data = open(mgf_path_dir+fn).readlines()
                ind = 0

            if data[ind] == 'BEGIN IONS\n':
                cnt = 0

            else:
                cnt += 1
                if cnt == 5:
                    spec_scan = int(data[ind+4].split('=')[1].split('\n')[0])
                    spec_fn = data[ind].split('=')[1].split('\n')[0]
                    spec_key = (spec_fn, spec_scan)

                    if d_scan > spec_scan:
                        ind += 1


                    if key == spec_key:
                        arr_mz = np.array([])
                        arr_int = np.array([])

                if key == spec_key and cnt > 12:
                    if data[ind] == 'END IONS\n' or data[ind] == 'END IONS':
                        b_ion, y_ion, num = residue(pept)
                        ion_list, neutral_list = res_ion(b_ion, y_ion, int(charge))
                        spec_mzarr, spec_intarr = remove_spec(arr_mz, arr_int, ion_list)
                        residue_mass, internal_list = frag_ion(pept, int(charge), num)
                        find_mass, find_int = find_spec(spec_mzarr, spec_intarr, residue_mass, neutral_list, internal = True)
                        find_list = []

                        if len(find_mass) != 0:
                            k = 0
                            while k <= len(internal_list)-1:
                                for j in range(len(find_mass)):
                                    temp_list = []
                                    if internal_list[k][2] == find_mass[j]:
                                        temp_list.append(ids)
                                        temp_list.append(pept)
                                        temp_list.append(internal_list[k][1])
                                        temp_list.append(find_int[j])
                                        find_list += [temp_list]
                                k += 1
                        else:
                            find_list += [[ids, pept, 0, 0]]

                        internal_fragment_ion += find_list
                        ind += 1
                        break

                    else:
                        mz = float(data[ind].split()[0])
                        intensity = float(data[ind].split()[1].split('\n')[0])
                        arr_mz = np.append(arr_mz, np.array([mz]))
                        arr_int = np.append(arr_int, np.array([intensity]))

            ind += 1

    end = time.time()
    spend = end - start

    internalDF = pd.DataFrame(internal_fragment_ion, columns = ['id', 'pept', 'ind', 'intensity'])

    print()
    print(spend/60)

    return internalDF

In [24]:
delta = pd.read_csv('groupbypeptide.tsv', sep = '\t')
delta = delta.loc[:, ['sequence', 'modified_sequence', 'PSM_ID', '#SpecFile', 'opt_global_originalfilename', 'nativeID', 'nativeID_scan', 'charge', 'baseFilename']]
delta = delta[(delta['charge']<7)&(delta['charge']>1)]
print(len(delta))
delta = delta[delta.sequence.apply(lambda x: len(str(x)) < 31)]
print(len(delta))
delta = delta.sort_values(['baseFilename', 'nativeID_scan']).reset_index(drop = True)
delta['id'] = delta.index
delta.head()

2107958
2000222


,sequence,PSM_ID,#SpecFile,opt_global_originalfilename,nativeID,nativeID_scan,charge,baseFilename,modified_sequence_li,id
0,AAAADSFSGGPAGVRLPR,949951,MSV000081142/peak/filtered_library_mgf_files/0...,spectrum_library_mgf_splits/72.mgf,scan=1,1,3,0112253cdebc4add94d374dfe279f900,+42.010565AAAADSFSGGPAGVRLPR,0
1,AAAEGPVGDGELWQTWLPNHVVFLR,10986,MSV000081142/peak/filtered_library_mgf_files/0...,spectrum_library_mgf_splits/72.mgf,scan=2,2,2,0112253cdebc4add94d374dfe279f900,+42.010565AAAEGPVGDGELWQTWLPNHVVFLR,1
2,AADALEEQQR,12356,MSV000081142/peak/filtered_library_mgf_files/0...,spectrum_library_mgf_splits/72.mgf,scan=3,3,2,0112253cdebc4add94d374dfe279f900,+42.010565AADALEEQQR,2
3,AAFRDIEEVSQGLLSLLGANR,952473,MSV000081142/peak/filtered_library_mgf_files/0...,spectrum_library_mgf_splits/72.mgf,scan=4,4,3,0112253cdebc4add94d374dfe279f900,+42.010565AAFRDLEEVSQGLLSLLGANR,3
4,AALGPSSQNVTEYVVRVP,952937,MSV000081142/peak/filtered_library_mgf_files/0...,spectrum_library_mgf_splits/72.mgf,scan=5,5,3,0112253cdebc4add94d374dfe279f900,+42.010565AALGPSSQNVTEYVVRVP,4


In [ ]:
delta_frac = delta.iloc[:1000000, :]
len(delta_frac)

In [17]:
internalDF = internal_fraction(delta_frac)

0 spectrum / 900000 spectrum
50000 spectrum / 900000 spectrum
100000 spectrum / 900000 spectrum
150000 spectrum / 900000 spectrum
200000 spectrum / 900000 spectrum
250000 spectrum / 900000 spectrum
300000 spectrum / 900000 spectrum
350000 spectrum / 900000 spectrum
400000 spectrum / 900000 spectrum
450000 spectrum / 900000 spectrum
500000 spectrum / 900000 spectrum
550000 spectrum / 900000 spectrum
600000 spectrum / 900000 spectrum
650000 spectrum / 900000 spectrum
700000 spectrum / 900000 spectrum
750000 spectrum / 900000 spectrum
800000 spectrum / 900000 spectrum
850000 spectrum / 900000 spectrum

39.79707710345586


In [18]:
internalDF.head()

,id,pept,ind,intensity_sum
0,100000,AMGKEDFTKIPHGVSGVQDRMSVIWERGVVGGK,0,1.183825e+07
1,100001,AMHTPKPAVGEEKDINTFLGTPVQK,0,2.178681e+06
2,100002,AMLKDSGPLFNTDYDILK,0,1.093537e+07
3,100003,AMMYSGELKFEKR,0,1.105971e+06
4,100004,AMNFADIERTITELGEK,0,3.066165e+06


In [20]:
internalDF.to_csv('0905by_100000.csv', index = False)